In [1]:
from cashe import get_train_test_split, get_input, get_training_inputs
from src.model_train import ModelTrainer, encode_label
from src.data_application import train_columns
from src.lstm_encoder import LSTMEmbedding
from src.data_local import MergedDataPreprocessing
import pandas as pd

In [2]:
df_train, df_test = get_train_test_split()

In [3]:
df_train.head(3)

,item_CreatedDate,transaction_RequestId,transaction_PatientAge,transaction_PatientEnGender,item_NameEn,item_Price,item_Status,item_Sequence,item_RequestQuantity,item_ResponseQuantity,transaction_DiagnosisIds,transaction_PhysicianIds,item_ResponseState
0,2023-08-24 15:03:23.404,c4427a02-3291-422d-b2e8-b97bf3d5081d,6,1,Examination,80.00,2,1,1.0,1.0,7,",327",approved
1,2023-08-29 11:48:54.060,e6f4ad37-13b9-4694-a61b-495384fcd245,28,0,Examination,80.00,6,1,1.0,0.0,17,",139",rejected
2,2023-08-29 11:48:54.060,e6f4ad37-13b9-4694-a61b-495384fcd245,28,0,PREGNANCY TEST (BLOOD),175.45,6,2,1.0,0.0,17,",139",rejected


In [4]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train, df_test)

In [5]:
X_train.head(3)

,transaction_PatientAge,transaction_PatientEnGender,item_NameEn,item_Price,item_Sequence,item_RequestQuantity,transaction_DiagnosisIds
0,6,1,Examination,80.00,1,1.0,7
1,28,0,Examination,80.00,1,1.0,17
2,28,0,PREGNANCY TEST (BLOOD),175.45,2,1.0,17


In [6]:
y_test[:5]

[0, 1, 0, 0, 0]

In [7]:
lstm_embedding = LSTMEmbedding()
data_prep = MergedDataPreprocessing(df_train)

X_train = data_prep.column_embedding(X_train,textual_col='item_NameEn')
X_test  = data_prep.column_embedding(X_test,textual_col='item_NameEn')

In [9]:
X_train

,transaction_PatientAge,transaction_PatientEnGender,item_NameEn,item_Price,item_Sequence,item_RequestQuantity,transaction_DiagnosisIds,item_NameEn1,item_NameEn2,item_NameEn3,...,item_NameEn7,item_NameEn8,item_NameEn9,item_NameEn10,item_NameEn11,item_NameEn12,item_NameEn13,item_NameEn14,item_NameEn15,item_NameEn16
0,6,1,Examination,80.00,1,1.0,7,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28,0,Examination,80.00,1,1.0,17,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28,0,PREGNANCY TEST (BLOOD),175.45,2,1.0,17,182,83,8,...,0,0,0,0,0,0,0,0,0,0
3,56,0,Examination,80.00,1,1.0,14,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,56,0,PERFALGAN,110.00,2,1.0,14,216,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,29,1,REPARIL 20 MG 40 DRAGEES,17.92,11,1.0,14,448,71,6,...,0,0,0,0,0,0,0,0,0,0
14994,0,1,"IRON BINDING CAPACITY, TOTAL (TIBC)",220.00,1,1.0,1,198,329,330,...,0,0,0,0,0,0,0,0,0,0
14995,0,1,"IRON, TOTAL (SERUM)",108.33,2,1.0,1,198,107,57,...,0,0,0,0,0,0,0,0,0,0
14996,0,1,FERRITIN,211.25,3,1.0,1,254,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
columns_needed = ['transaction_PatientAge','transaction_PatientEnGender','item_Price','item_Sequence','item_RequestQuantity','transaction_DiagnosisIds','item_NameEn1','item_NameEn2','item_NameEn3','item_NameEn4','item_NameEn5','item_NameEn6','item_NameEn7','item_NameEn8','item_NameEn9', 'item_NameEn10','item_NameEn11', 'item_NameEn12', 'item_NameEn13', 'item_NameEn14','item_NameEn15', 'item_NameEn16']

X_train = X_train[columns_needed]
X_test  = X_test[columns_needed]

In [11]:
X_train

,transaction_PatientAge,transaction_PatientEnGender,item_Price,item_Sequence,item_RequestQuantity,transaction_DiagnosisIds,item_NameEn1,item_NameEn2,item_NameEn3,item_NameEn4,...,item_NameEn7,item_NameEn8,item_NameEn9,item_NameEn10,item_NameEn11,item_NameEn12,item_NameEn13,item_NameEn14,item_NameEn15,item_NameEn16
0,6,1,80.00,1,1.0,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28,0,80.00,1,1.0,17,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28,0,175.45,2,1.0,17,182,83,8,0,...,0,0,0,0,0,0,0,0,0,0
3,56,0,80.00,1,1.0,14,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,56,0,110.00,2,1.0,14,216,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,29,1,17.92,11,1.0,14,448,71,6,196,...,0,0,0,0,0,0,0,0,0,0
14994,0,1,220.00,1,1.0,1,198,329,330,107,...,0,0,0,0,0,0,0,0,0,0
14995,0,1,108.33,2,1.0,1,198,107,57,0,...,0,0,0,0,0,0,0,0,0,0
14996,0,1,211.25,3,1.0,1,254,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
model_runner = ModelTrainer(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)

In [11]:
model_runner.train_models()
res = model_runner.evaluate_models()

[LightGBM] [Info] Number of positive: 7480, number of negative: 7518
[LightGBM] [Info] Total Bins 1303
[LightGBM] [Info] Number of data points in the train set: 14998, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498733 -> initscore=-0.005067
[LightGBM] [Info] Start training from score -0.005067


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [13]:
old_train_columns = \
    ['transaction_PatientAge','transaction_PatientEnGender','item_Price','item_Sequence','item_RequestQuantity','transaction_DiagnosisIds']

model_runner_old = ModelTrainer(X_train=X_train[old_train_columns],y_train=y_train,X_test=X_test[old_train_columns],y_test=y_test)
model_runner_old.train_models()

res_before = model_runner_old.evaluate_models()

[LightGBM] [Info] Number of positive: 7480, number of negative: 7518
[LightGBM] [Info] Total Bins 359
[LightGBM] [Info] Number of data points in the train set: 14998, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498733 -> initscore=-0.005067
[LightGBM] [Info] Start training from score -0.005067


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [21]:
len(df_test)

3723

In [15]:
## Comparison between the embedding vector added columns
print('The result before embedding the Item_Name column are:\n',res_before,'\nThe results after the embedding:\n',res)

The result before embedding the Item_Name column are:
 {'Decision Tree': {'Accuracy': 0.67, 'Precision': 0.66, 'Recall': 0.64}, 'LightGBM': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.71}, 'SGD Classifier': {'Accuracy': 0.52, 'Precision': 0.54, 'Recall': 0.01}, 'XGBoost': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.73}, 'Neural Network': {'Accuracy': 0.57, 'Precision': 0.59, 'Recall': 0.33}} 
The results after the embedding:
 {'Decision Tree': {'Accuracy': 0.7, 'Precision': 0.69, 'Recall': 0.67}, 'LightGBM': {'Accuracy': 0.74, 'Precision': 0.71, 'Recall': 0.77}, 'SGD Classifier': {'Accuracy': 0.49, 'Precision': 0.49, 'Recall': 0.99}, 'XGBoost': {'Accuracy': 0.74, 'Precision': 0.71, 'Recall': 0.78}, 'Neural Network': {'Accuracy': 0.65, 'Precision': 0.7, 'Recall': 0.5}}


In [16]:
## Feature importance Case Study
dt_tran = model_runner.get_decision_tree_feature_importance()
dt_feats = pd.DataFrame(list(dt_tran),columns_needed)

dt_feats[0].sort_values(ascending=False)

transaction_PatientAge         0.270896
item_NameEn1                   0.175273
item_Sequence                  0.154727
item_Price                     0.126149
transaction_DiagnosisIds       0.090805
item_NameEn2                   0.045057
transaction_PatientEnGender    0.040445
item_NameEn3                   0.037848
item_NameEn5                   0.021284
item_NameEn4                   0.014264
item_NameEn7                   0.011622
item_RequestQuantity           0.005194
item_NameEn6                   0.003837
item_NameEn9                   0.000806
item_NameEn15                  0.000429
item_NameEn13                  0.000338
item_NameEn12                  0.000322
item_NameEn8                   0.000287
item_NameEn11                  0.000219
item_NameEn10                  0.000102
item_NameEn14                  0.000097
item_NameEn16                  0.000000
Name: 0, dtype: float64

In [17]:
gbm_tran = model_runner.get_lightgbm_feature_importance()
gbm_feats = pd.DataFrame(list(gbm_tran),columns_needed)

gbm_feats[0].sort_values(ascending=False)

item_Price                     0.236333
transaction_PatientAge         0.170000
item_NameEn1                   0.112333
transaction_DiagnosisIds       0.090000
item_NameEn2                   0.081667
item_Sequence                  0.080667
item_NameEn3                   0.080000
item_NameEn4                   0.052000
item_NameEn5                   0.033667
transaction_PatientEnGender    0.028333
item_RequestQuantity           0.013333
item_NameEn6                   0.012000
item_NameEn7                   0.006333
item_NameEn8                   0.002333
item_NameEn9                   0.000667
item_NameEn10                  0.000333
item_NameEn11                  0.000000
item_NameEn12                  0.000000
item_NameEn13                  0.000000
item_NameEn14                  0.000000
item_NameEn15                  0.000000
item_NameEn16                  0.000000
Name: 0, dtype: float64

In [18]:
nn_tran = model_runner.get_neural_network_feature_importance()
nn_feats = pd.DataFrame(list(nn_tran),columns_needed)

nn_feats[0].sort_values(ascending=False)

item_Sequence                   1.350773e-01
item_NameEn8                    9.657824e-02
transaction_PatientAge          7.717226e-02
item_NameEn2                    7.321714e-02
item_Price                      7.049322e-02
transaction_PatientEnGender     7.037843e-02
item_NameEn6                    6.129502e-02
item_NameEn7                    4.200646e-02
transaction_DiagnosisIds        2.363682e-02
item_NameEn3                    2.121282e-02
item_NameEn4                    1.826765e-02
item_NameEn1                    1.813903e-02
item_RequestQuantity            3.366400e-03
item_NameEn5                    1.837944e-04
item_NameEn16                   2.861594e-65
item_NameEn13                   2.758648e-68
item_NameEn15                   5.924836e-84
item_NameEn10                   2.077948e-89
item_NameEn14                   2.426870e-92
item_NameEn11                   5.443403e-95
item_NameEn9                    1.038341e-97
item_NameEn12                  4.883545e-103
Name: 0, d

## K-Fold Analysis

In [2]:
df_original = get_input()
preprocessing = MergedDataPreprocessing(df_original)

In [3]:
from src.model_train import validate_folds

df_acc = validate_folds(df_original)

E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7490, number of negative: 7588
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 15078, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.496750 -> initscore=-0.012999
[LightGBM] [Info] Start training from score -0.012999


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7437, number of negative: 7506
[LightGBM] [Info] Total Bins 1303
[LightGBM] [Info] Number of data points in the train set: 14943, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497691 -> initscore=-0.009235
[LightGBM] [Info] Start training from score -0.009235


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7480, number of negative: 7714
[LightGBM] [Info] Total Bins 1317
[LightGBM] [Info] Number of data points in the train set: 15194, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.492300 -> initscore=-0.030804
[LightGBM] [Info] Start training from score -0.030804


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7558, number of negative: 7626
[LightGBM] [Info] Total Bins 1298
[LightGBM] [Info] Number of data points in the train set: 15184, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497761 -> initscore=-0.008957
[LightGBM] [Info] Start training from score -0.008957


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, col] = df2[col].values
E:\Projects\Claims_Rejection\src\data_local.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

[LightGBM] [Info] Number of positive: 7480, number of negative: 7518
[LightGBM] [Info] Total Bins 1300
[LightGBM] [Info] Number of data points in the train set: 14998, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498733 -> initscore=-0.005067
[LightGBM] [Info] Start training from score -0.005067


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [4]:
df_acc

,Decision Tree,LightGBM,SGD Classifier,XGBoost,Neural Network
0,67,72,50,72,67
1,67,72,46,72,69
2,67,70,51,72,67
3,69,72,49,74,64
4,68,72,52,72,67


In [ ]:
## XGBoost 72.5% accuracy, Neural Networks 66%  accuracy which can definitely be improved